In [1]:
import pandas as pd
import numpy as np
import re
import time
import json
import glob
import tqdm

In [2]:
# active leaning - é muito caro fazer anotações em novos exemplos
df = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df=df[df['y'].notnull()]

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [4]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

In [6]:
clean_date= df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x if len(x)==1 else x)
mapa_meses = {"jan":"Jan", "fev":"Feb", "mar":"Mar", "abr":"Apr", "mai":"May", "jun":"Jun","jul":"Jul", "ago":"Aug", "set":"Sep","out":"Oct","nov":"Nov","dez":"Dec"}
clean_date[1] = clean_date[1].map(mapa_meses)
clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

In [7]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)
df_limpo['views'] =views

<ipython-input-7-86d1434b4248>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)


In [13]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [14]:
features['tempo_desd_pub'] = (pd.to_datetime("2021-02-13") - df_limpo['date'])/np.timedelta64(1, "D")
features['views'] = df_limpo['views']
features['views_por_dias'] = features['views'] / features['tempo_desd_pub']
features = features.drop(['tempo_desd_pub'], axis =1)

In [16]:
features.head()

,views,views_por_dias
0,28028,31.351230
1,1131,1.379268
2,1816,2.781011
3,1171,2.129091
4,1228,1.523573


In [18]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = df_limpo['date'] >= "2019-04-01"
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (270, 2), (228,), (270,))

In [97]:
# da msi peso mais peso para palavras q aparecem muito em determinado exemplo mas nao aparece tanto pro dataset
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

In [98]:
title_train[7]

'Machine Learning In 5 Minutes | Machine Learning Introduction |What Is Machine Learning |Simplilearn'

In [99]:
#https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76
#min_df numero mínimo que uma palavra precisa a aparecer no dados para que se torne uma coluna pelo menos duas vezes a palavra apareceu 
title_vec= TfidfVectorizer(min_df=2)

In [101]:
# quais as palavras que viu e vai guardar a frequencia delas no dataset
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [73]:
X_words = title_vec.get_feature_names()

In [41]:
# matriz espaça armazenar somente quanto for diferente de zero
# das 193 palavas que aparecem pelo menos 2 vezes qual a frequencia delas em cada video
print(title_bow_train.shape)

(228, 193)


In [116]:
from scipy.sparse import hstack, vstack
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [117]:
print(Xtrain_wtitle.shape)

(228, 195)


In [118]:
mdl=RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [119]:
p=mdl.predict_proba(Xval_wtitle)[:,1]

In [120]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [121]:
p.shape

(270,)

In [122]:
average_precision_score(yval, p)

0.18465551413200237

In [123]:
roc_auc_score(yval,p)

0.5773252279635258